# Analyzing ad selection

Ads that run in free Android and iOS mobile apps available on Google Play and the App Store are our main source of revenue. To assist our developers to enhance customer experience and increase revenue to our company, an analysis of what content our customers prefer was conducted.

## Downloading The Data
The Google Play data set contains approximately 10,000 Android apps; the data was collected in August 2018. You can download the data set [here](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).

The App Store data set contains approximately 7,000 iOS apps; the data was collected in July 2017. You can download the data set [here](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

## Import  and Explore Data

In [1]:
#import function
from csv import reader
import pandas as pd
#define function
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
#Open and Explore the Google Play data set
opened_file = open('googleplaystore.csv', encoding="utf8")
read_file = reader(opened_file)
google = list(read_file)
google_header = google[0]
google = google[1:]

explore_data(google, 0, 4, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13


In [3]:
#Open and Explore the App Store data set
opened_file = open('AppleStore.csv', encoding="utf8")
read_file = reader(opened_file)
apple = list(read_file)
apple_header = apple[0]
apple = apple[1:]

explore_data(apple, 0, 4, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


In [4]:
#inspect column names to identify which columns are most important
print(google_header)
print(apple_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


## Data Preparation

In [5]:
#detect row that is not the same length as the header row
#if all rows are the same, there will be no output
for row in google:
    header_length = len(google_header)
    row_length = len(row)
    if row_length != header_length:
        print(row)
        print(google.index(row))
  
#delete row
del google[10472]

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10472


In [6]:
#detect row that is not the same length as the header row
#if all rows are the same, there will be no output
for row in apple:
    header_length = len(apple_header)
    row_length = len(row)
    if row_length != header_length:
        print(row)
        print(apple.index(row))

In [7]:
#detect duplicates
google_duplicate_apps = []
google_duplicate_indices = []
google_unique_apps = []

for row in google:
    name = row[0]
    if name in google_unique_apps:
        google_duplicate_apps.append(name)            
    else:
        google_unique_apps.append(name)
    
print('Number of duplicate apps:', len(google_duplicate_apps))

Number of duplicate apps: 1181


In [8]:
#create for loop to eliminate duplicate apps from Google Play Store data set
reviews_max = {}

for row in google:
    name = row[0]
    n_reviews = float(row[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
    
print('Number of duplicate apps:', len(reviews_max))

Number of duplicate apps: 9659


First, two empty lists are created: `google_clean` and `already_added`. The for loop below will go through the Google data set and put the apps' name in the `already_added` list and the apps' updated number of reviews in the `google_clean` list. 

In [9]:

google_clean = []
already_added = []

for row in google:
    app_name = row[0]
    n_reviews = float(row[3])
    
    if reviews_max[app_name] == n_reviews and app_name not in already_added:
        google_clean.append(row)
        already_added.append(app_name)
        
print('Number of duplicate apps:', len(google_clean))

Number of duplicate apps: 9659


In [10]:
#detect duplicates
duplicate_apps = []
unique_apps = []

for app in apple:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))

Number of duplicate apps: 0


Our company wants to focus on English only apps. Let's detect and eliminate non-English apps in either data set. 

English characters correspond to numbers less than 127. The scan function will tell the user if the string is English by returning True or Non-English by returning False. 

Some examples are included to test the function.

In [11]:
def is_english(string):
    for character in string:
        if ord(character) > 127:
            return False
        elif ord(character) < 127:
            return True
    
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('😜'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False
False


The function is not evaluating emojis correctly so improvements to the function are written below. The previous examples are used to test the function.

In [12]:
def is_english(string):
    for character in string:
        non_ascii = 0
        if ord(character) > 127:
            non_ascii += 1
            
        if non_ascii > 3:
            return False
        else:
            return True
    
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('😜'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
True
True


The is_english function will check that all apps in either data set contain only english apps. Explore each data set.

In [13]:
google_english_apps = []

for row in google_clean:
    app_name = row[0]
    if is_english(app_name) == True:
        google_english_apps.append(row)
         
explore_data(google_english_apps, 0, 4, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 9659
Number of columns: 13


In [14]:
apple_english_apps = []

for row in apple:
    app_name = row[1]
    if is_english(app_name)== True:
        apple_english_apps.append(row)
        
explore_data(apple_english_apps, 0, 4, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


Since the company is only interested in the free apps. Isolate the free apps from the Google Play Store data set and the Apple Store data sets.

In [15]:
#isolate the free apps from the Google Play Store data set
free_google_english_apps = []

for row in google_english_apps:
    price = row[7]
    if price == '0':
        free_google_english_apps.append(row)

#print the length of the Google Play Store data set      
print(len(free_google_english_apps))

8905


In [16]:
#isolate the free apps from the App Store data set
free_apple_english_apps = []

for row in apple_english_apps:
    price = row[4]
    
    if price == '0.0':
        free_apple_english_apps.append(row)

#print the length of the App Store data set      
print(len(free_apple_english_apps))

4056


## Aim and validation strategy of the analysis

The aim of the analysis is to determine the kinds of apps that are likely to attract more users so that the company's revenue will increase.

To minimize risks and overhead, the validation strategy for an app idea is comprised of three steps:

* Build a minimal Android version of the app, and add it to Google Play.
* If the app has a good response from users, develop it further
* If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

## Build frequency tables for the most common genres for apps in Google Play and App Store

The most common genres are `prime_genre`, `Genres`, and `Category`. Two function will be used to analyze the frequency tables. One function to generate the frequency tables that show percentages. Another function to display the percentages in descending order.

In [17]:
#define a function to create a frequency table
def freq_table(dataset, index):
    table = {}
    total = 0
    #build frequency table
    for row in dataset:
        total += 1
        app = row[index]
        if app in table:
            table[app] += 1
        else:
            table[app] = 1
    #convert the frequecy from integers to decimals
    table_percentages = {}
    for variable in table:
        percentage = (table[variable] / total) * 100
        #assign apps to their percetages
        table_percentages[variable] = percentage
    return table_percentages

#define a function to display a frequency table
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Let's build the frequency tables for both data sets!

In [18]:
#create a genres frequency table using Google Play Store data set
google_genres = freq_table(free_google_english_apps, -4)

#create a category frequency table using Google Play Store data set
google_category = freq_table(free_google_english_apps, 1)

In [19]:
#create a prime_genre frequency table using App Store data set
apple_freq_table = freq_table(free_apple_english_apps, -5)

Now, focus on analyzing the three frequency tables.

In [20]:
#display the genres column in the Google Play Store data set
display_table(free_google_english_apps, -4)

Tools : 8.422234699606962
Entertainment : 6.086468276249298
Education : 5.390230207748456
Business : 4.581695676586187
Lifestyle : 3.9191465468837734
Productivity : 3.885457608085345
Finance : 3.6833239752947784
Medical : 3.5148792813026386
Sports : 3.4475014037057834
Personalization : 3.312745648512072
Communication : 3.2341381246490735
Action : 3.0881527231892196
Health & Fitness : 3.065693430656934
Photography : 2.9421673217293653
News & Magazines : 2.829870859067939
Social : 2.6501965188096577
Travel & Local : 2.313307130825379
Shopping : 2.2459292532285233
Books & Reference : 2.1785513756316677
Simulation : 2.0662549129702414
Dating : 1.8528916339135317
Arcade : 1.8416619876473892
Video Players & Editors : 1.7742841100505335
Casual : 1.7518248175182483
Maps & Navigation : 1.4149354295339696
Food & Drink : 1.235261089275688
Puzzle : 1.1229646266142617
Racing : 0.9882088714205502
Role Playing : 0.9320606400898372
Libraries & Demo : 0.9320606400898372
Strategy : 0.9208309938236946
Au

In [21]:
#display the category column in the Google Play Store data set
display_table(free_google_english_apps, 1)

FAMILY : 18.97810218978102
GAME : 9.70241437394722
TOOLS : 8.433464345873105
BUSINESS : 4.581695676586187
LIFESTYLE : 3.9303761931499155
PRODUCTIVITY : 3.885457608085345
FINANCE : 3.6833239752947784
MEDICAL : 3.5148792813026386
SPORTS : 3.3801235261089273
PERSONALIZATION : 3.312745648512072
COMMUNICATION : 3.2341381246490735
HEALTH_AND_FITNESS : 3.065693430656934
PHOTOGRAPHY : 2.9421673217293653
NEWS_AND_MAGAZINES : 2.829870859067939
SOCIAL : 2.6501965188096577
TRAVEL_AND_LOCAL : 2.3245367770915215
SHOPPING : 2.2459292532285233
BOOKS_AND_REFERENCE : 2.1785513756316677
DATING : 1.8528916339135317
VIDEO_PLAYERS : 1.7967434025828188
MAPS_AND_NAVIGATION : 1.4149354295339696
FOOD_AND_DRINK : 1.235261089275688
EDUCATION : 1.167883211678832
ENTERTAINMENT : 0.9545199326221224
LIBRARIES_AND_DEMO : 0.9320606400898372
AUTO_AND_VEHICLES : 0.9208309938236946
HOUSE_AND_HOME : 0.8197641774284109
WEATHER : 0.7973048848961257
EVENTS : 0.7074677147669848
PARENTING : 0.6513194834362718
ART_AND_DESIGN : 0

In [22]:
#display the prime_genre column in the Google Play Store data set
display_table(free_apple_english_apps, -5)

Games : 55.64595660749507
Entertainment : 8.234714003944774
Photo & Video : 4.117357001972387
Social Networking : 3.5256410256410255
Education : 3.2544378698224854
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597637
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583828
Book : 1.6272189349112427
Productivity : 1.5285996055226825
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613412
Reference : 0.4930966469428008
Navigation : 0.4930966469428008
Business : 0.4930966469428008
Catalogs : 0.22189349112426035
Medical : 0.19723865877712032


# Calculating the Most Popular Apps by Genre from App Store

In [23]:
#create a prime_genre frequency table using App Store data set
genre_apple = freq_table(free_apple_english_apps, -5)

#loop over unique genres in App Store with interation variable 'genre'
for genre in genre_apple:
    #initiate 'total' varaiable with value 0
    total = 0
    #initiate 'len_genre' variable with value 0
    len_genre = 0
    
    #loop over preprocessed App Store data set with interation variable 'app'
    for app in free_apple_english_apps:
        #save app genre to 'genre_app' variable
        genre_app = app[-5]
        #condtional statement
        #if the genre column in the frequecy table matches the preprocessed App Store data set
        if genre_app == genre:
            #save the number of user ratings as a float
            user_rating = float(app[5])
            total += user_rating
            len_genre += 1
    #calculate the average for each genre        
    avg = total / len_genre
    print(genre, ':', avg)
    

Social Networking : 53078.195804195806
Photo & Video : 27249.892215568863
Games : 18924.68896765618
Music : 56482.02985074627
Reference : 67447.9
Health & Fitness : 19952.315789473683
Weather : 47220.93548387097
Utilities : 14010.100917431193
Travel : 20216.01785714286
Shopping : 18746.677685950413
News : 15892.724137931034
Navigation : 25972.05
Lifestyle : 8978.308510638299
Entertainment : 10822.961077844311
Food & Drink : 20179.093023255813
Sports : 20128.974683544304
Book : 8498.333333333334
Finance : 13522.261904761905
Education : 6266.333333333333
Productivity : 19053.887096774193
Business : 6367.8
Catalogs : 1779.5555555555557
Medical : 459.75


# Calculating the Most Popular Apps by Genre from the Google Play Store

In [24]:
#create a category frequency table using Google Play Store data set
category_google = freq_table(free_google_english_apps, 1)

#loop over unique categories in Google Play Store with interation variable 'category'
for category in category_google:
    #initiate 'total' varaiable with value 0
    total = 0
    #initiate 'len_category' variable with value 0
    len_category = 0
    
    #loop over preprocessed Google Play Store data set with interation variable 'app'
    for app in free_google_english_apps:
        #save app category to 'category_app' variable
        category_app = app[1]
        #condtional statement
        #if the category column in the frequecy table matches the preprocessed Google Play Store data set
        if category_app == category:
            #save the number of installs
            n_install = app[5]
            #remove commas
            n_install = n_install.replace(',', '')
            #remove plus signs 
            n_install = n_install.replace('+', '')
            total += float(n_install)
            len_category += 1
    #calculate the average for each category        
    avg = total / len_category
    print(category, ':', avg)

ART_AND_DESIGN : 1952105.1724137932
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8587351.855670104
BUSINESS : 1708215.906862745
COMICS : 803234.8214285715
COMMUNICATION : 38322625.697916664
DATING : 854028.8303030303
EDUCATION : 1825480.7692307692
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1436126.94
GAME : 15551995.891203703
FAMILY : 3668870.823076923
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7001693.425
PHOTOGRAPHY : 17772018.759541985
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10787009.952063914
PERSONALIZATION : 5183850.806779661
PRODUCTIVITY : 16738957.554913295
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24573948.25
NEWS_AND_MAGAZINES : 9401635.95